In [ ]:
from huggingface_hub import login
login(token="XXXXXXXXXXXXXXXXXXXXXXXXXX")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\User\.cache\huggingface\token
Login successful


In [15]:
from datasets import load_dataset, Dataset
from huggingface_hub import HfApi, HfFolder

# Define your local dataset path and the Hugging Face repository ID
local_dataset_path = "new_dataset.parquet"
hf_repo_id = "JanDkff/TinyFuncCoder"

dataset = load_dataset('parquet', data_files=local_dataset_path, split='train', streaming=True)

# Initialize Hugging Face API
api = HfApi()

# Log in to Hugging Face
# Note: You should have your Hugging Face token set in the environment or you can login using the HfApi method
token = HfFolder.get_token()
if token is None:
    raise ValueError("You need to login to Hugging Face first. Use `HfApi().login(token)` to set your token.")

# Initialize an empty list to store the data in chunks
data_chunks = []

# Function to convert list of dictionaries to dictionary of lists
def list_of_dicts_to_dict_of_lists(data_list):
    dict_of_lists = {}
    for d in data_list:
        for key, value in d.items():
            if key not in dict_of_lists:
                dict_of_lists[key] = []
            dict_of_lists[key].append(value)
    return dict_of_lists

# Process the dataset in chunks
chunk_size = 2500000  # Adjust this based on your memory constraints

j = 0
for i, example in enumerate(dataset):
    example['params'] = str(example['params'])
    data_chunks.append(example)
    
    # Once we reach the chunk size, create a dataset and push it to Hugging Face
    if (i + 1) % chunk_size == 0:
        temp_dataset = Dataset.from_dict(list_of_dicts_to_dict_of_lists(data_chunks))
        temp_dataset.push_to_hub(hf_repo_id, token=token, split=f"part_{j}")
        print(j)
        j += 1
        data_chunks = []  # Reset the data chunks
        temp_dataset = None

# Push the remaining data
if data_chunks:
    temp_dataset = Dataset.from_dict(list_of_dicts_to_dict_of_lists(data_chunks))
    temp_dataset.push_to_hub(hf_repo_id, token=token, split=f"part_{j}")
    data_chunks = []  # Reset the data chunks
    temp_dataset = None


Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/834 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/834 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/834 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

...

5


Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/737 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/737 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

In [16]:
from datasets import load_dataset, Dataset, Features, Value, Sequence
from huggingface_hub import HfApi, HfFolder

# Define your local dataset path and the Hugging Face repository ID
local_dataset_path = "new_dataset.json"
hf_repo_id = "JanDkff/TinyFuncCoder"

# Define the expected features schema
expected_features = Features({
    'name': Value(dtype='string'),
    'params': Value(dtype='string'),
    'body': Value(dtype='string'),
    'docstring': Value(dtype='string'),
    'file_id': Value(dtype='string'),
    'language': Value(dtype='string')
})

# Load the dataset with streaming
dataset = load_dataset('json', data_files=local_dataset_path, split='train', streaming=True)

# Initialize Hugging Face API
api = HfApi()

# Log in to Hugging Face
token = HfFolder.get_token()
if token is None:
    raise ValueError("You need to login to Hugging Face first. Use `HfApi().login(token)` to set your token.")

# Initialize an empty list to store the data in chunks
data_chunks = []

# Function to convert list of dictionaries to dictionary of lists
def list_of_dicts_to_dict_of_lists(data_list):
    dict_of_lists = {}
    for d in data_list:
        for key, value in d.items():
            if key not in dict_of_lists:
                dict_of_lists[key] = []
            dict_of_lists[key].append(value)
    return dict_of_lists

# Process the dataset in chunks
chunk_size = 2500000  # Adjust this based on your memory constraints

j += 1
for i, example in enumerate(dataset):
    data_chunks.append(example)
    
    # Once we reach the chunk size, create a dataset and push it to Hugging Face
    if (i + 1) % chunk_size == 0:
        temp_dataset = Dataset.from_dict(list_of_dicts_to_dict_of_lists(data_chunks), features=expected_features)
        temp_dataset.push_to_hub(hf_repo_id, token=token, split=f"part_{j}")
        print(j)
        j += 1
        data_chunks = []  # Reset the data chunks
        temp_dataset = None

# Push the remaining data
if data_chunks:
    temp_dataset = Dataset.from_dict(list_of_dicts_to_dict_of_lists(data_chunks), features=expected_features)
    temp_dataset.push_to_hub(hf_repo_id, token=token, split=f"part_{j}")
    data_chunks = []  # Reset the data chunks
    temp_dataset = None


Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/625 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/625 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/625 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/625 [00:00<?, ?ba/s]

...

21


Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/813 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/813 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/7.24k [00:00<?, ?B/s]

In [81]:
from datasets import load_dataset
import os

In [ ]:
for path in os.listdir("checkdata"):
    data = load_dataset("parquet", data_files={'train': f'checkdata/{path}'})
    print(path.split('-')[0], data['train'].features)

Generating train split: 0 examples [00:00, ? examples/s]

part_0 {'name': Value(dtype='string', id=None), 'params': Value(dtype='string', id=None), 'body': Value(dtype='string', id=None), 'docstring': Value(dtype='string', id=None), 'file_id': Value(dtype='string', id=None), 'language': Value(dtype='string', id=None)}


Generating train split: 0 examples [00:00, ? examples/s]

part_1 {'name': Value(dtype='string', id=None), 'params': Value(dtype='string', id=None), 'body': Value(dtype='string', id=None), 'docstring': Value(dtype='string', id=None), 'file_id': Value(dtype='string', id=None), 'language': Value(dtype='string', id=None)}


Generating train split: 0 examples [00:00, ? examples/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

part_7 {'name': Value(dtype='string', id=None), 'params': Value(dtype='string', id=None), 'body': Value(dtype='string', id=None), 'docstring': Value(dtype='string', id=None), 'file_id': Value(dtype='string', id=None), 'language': Value(dtype='string', id=None)}


Generating train split: 0 examples [00:00, ? examples/s]

part_8 {'name': Value(dtype='string', id=None), 'params': Value(dtype='string', id=None), 'body': Value(dtype='string', id=None), 'docstring': Value(dtype='string', id=None), 'file_id': Value(dtype='string', id=None), 'language': Value(dtype='string', id=None)}


Generating train split: 0 examples [00:00, ? examples/s]